# MABP _ Multi Armed Bandit Problem

# Random Selection

In [1]:
# Random Selection

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('../input/bandit/Ads_Optimisation.csv')

# Implementing Random Selection
import random
N = 10000
d = 10
ads_selected = []
total_reward = 0
for n in range(0, N): 
    ad = random.randrange(d)
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    total_reward = total_reward + reward # total_reward  : 1251

MAB에서는 모든 행동이 순서대로 발생한다 가정.

그 순서에 따라 

시점 $t$의 행동을 $A_t$라 하고, 

행동에 따른 보상은 $R_t$

행동 $a$의 가치는 $q_*(a)$, 

시점 $t$에 추정된 가치는 $Q_t(a)$



In [2]:
pd.Series(ads_selected).tail(1000).value_counts(normalize = True)

9    0.121
0    0.113
5    0.111
1    0.105
3    0.099
8    0.098
6    0.096
2    0.090
7    0.086
4    0.081
dtype: float64

Total reward for the random selection algorithm comes out to be 1170. As this algorithm is not learning anything, it will not smartly select any ad which is giving the maximum return. And hence even if we look at the last 1000 trials, it is not able to find the optimal ad.

# UCB

입실론-그리디 알고리즘은 최적의 주식을 찾기 위해 랜덤하게 탐험합니다. 그러나 이 랜덤성으로 인해서 최적값과는 멀어지기도 하고 탐색을 과도하게 하는 문제가 있습니다. 예를 들어, 10개의 행동 중 2개가 최선의 산황이여도 입실론-그리디 알고리즘은 나머지 8개도 공평하게 탐색하기 때문에 과도하게 탐색하는 문제가 발생합니다.

이를 해결하기 위해 널리 쓰이는 방법 중 하나는 **UCB (Upper-Confidence-Bound) 알고리즘**입니다. 


**UCB (Upper-Confidence-Bound) 알고리즘**

**알고리즘의 아이디어 :**

     추정된 가치 $Q_t(a)$에서 일종의 신뢰 구간을 구해서 그 구간의 위쪽 신뢰 구간의 행동을 선택.

**수식 :**
$A_t = argmax_a [ Q_t(a) + c\sqrt{\frac{\ln t}{N_t(a)}}]$

- $t$는 현재 시점, 

- $N_t(a)$는 현재 시점까지 행동 $a$를 한 횟수

일반적으로 신뢰구간을 구하는 식 $\mu + c\sqrt{\sigma^2 / n}$에서 $\mu$대신 $Q_t(a)$, $\sigma^2$대신 $\ln t$를 사용한 것만 다름을 알 수 있다.

 - 여기서 상수 $c$는 신뢰구간 폭을 제어하는 파라미터

   -> $c$를 크게 하면 탐색을 많이 하게 되고 작게 하면 활용을 많이 하게 됨. 입실론-그리디 알고리즘에서 입실론이 크면 랜덤하게 탐색할 확률이 커지듯이 $c$도 값이 클수록 탐색을 많이 하게 됨.

 - $c\sqrt{\frac{\ln t}{N_t(a)}}$항을 보면, $N_t(a)$를 분모에 두어 충분히 탐험되지 않은 행동에 가중치를 줌.
 
   -> $N_t(a)$가 작을수록 $c\sqrt{\frac{\ln t}{N_t(a)}}$)은 커지기 때문이죠! 결국 UCB 알고리즘에 들어가는 첫 번째 항 $Q_t(a)$은 활용 (Exploitation)에 중점을 둔 항이고, 두 번째 항 $c\sqrt{\frac{\ln t}{N_t(a)}}$은 탐색 (Explotation)에 중점을 둔 항.
   ->UCB 알고리즘은 활용과 탐색을 적절히 고려해서 선택하는 알고리즘이라 할 수 있다.

In [3]:
# Implementing UCB
import math
N = 10000
d = 10
ads_selected = []
numbers_of_selections = [0] * d
sums_of_reward = [0] * d
total_reward = 0

for n in range(0, N):
    ad = 0
    max_upper_bound = 0
    for i in range(0, d):
        if (numbers_of_selections[i] > 0):
            average_reward = sums_of_reward[i] / numbers_of_selections[i]
            delta_i = math.sqrt(2 * math.log(n+1) / numbers_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            ad = i
    ads_selected.append(ad) #선택된 ad (t 시점에서 upper bound가장 큰거) 10000행 1열
    numbers_of_selections[ad] += 1 # 선택된 광고 댕긴 횟수 한 회 증가시키기
    # numbers_of_selections은 10개의 광고 있으니까 ; [947, 417, 338, 380, 5630, 180, 435, 1106, 352, 215]

    reward = dataset.values[n, ad]
    sums_of_reward[ad] += reward # sums_of_reward : [176, 48, 31, 40, 1519, 1, 52, 217, 34, 7]
    total_reward += reward # 2125

In [4]:
pd.Series(ads_selected).tail(1000).value_counts(normalize = True)

4    0.771
0    0.106
7    0.034
3    0.034
2    0.026
1    0.007
6    0.007
8    0.006
9    0.005
5    0.004
dtype: float64

After just 1500 trials, UCB is already favouring Ad #5 (index 4) which happens to be the optimal ad, and gets the maximum return for the given problem.

In [5]:
total_reward

2125

> ##  참고 한번에 합쳐 놓은 버전

In [6]:

print(dataset.head())
# Implementing Random Selection
print('\n\nRandom Selection\n\n')

import random
N = 10000
d = 10
ads_selected = []
total_reward = 0
for n in range(0, N):
    ad = random.randrange(d)
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    total_reward = total_reward + reward


print(pd.Series(ads_selected).tail(1000).value_counts(normalize=True))


# Implementing UCB
print('\n\nImplementing UCB\n\n')
import math
N = 10000
d = 10
ads_selected = []
numbers_of_selections = [0] * d
sums_of_reward = [0] * d
total_reward = 0

for n in range(0, N):
    ad = 0
    max_upper_bound = 0
    for i in range(0, d):
        if (numbers_of_selections[i] > 0):
            average_reward = sums_of_reward[i] / numbers_of_selections[i]
            delta_i = math.sqrt(2 * math.log(n+1) / numbers_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            ad = i
    ads_selected.append(ad)
    numbers_of_selections[ad] += 1
    reward = dataset.values[n, ad]
    sums_of_reward[ad] += reward
    total_reward += reward


print(pd.Series(ads_selected).head(1500).value_counts(normalize=True))

   Ad 1  Ad 2  Ad 3  Ad 4  Ad 5  Ad 6  Ad 7  Ad 8  Ad 9  Ad 10
0     1     0     0     0     1     0     0     0     1      0
1     0     0     0     0     0     0     0     0     1      0
2     0     0     0     0     0     0     0     0     0      0
3     0     1     0     0     0     0     0     1     0      0
4     0     0     0     0     0     0     0     0     0      0


Random Selection


8    0.125
3    0.109
5    0.102
4    0.101
0    0.099
2    0.099
7    0.099
9    0.098
1    0.087
6    0.081
dtype: float64


Implementing UCB


4    0.250000
7    0.170667
0    0.106000
3    0.091333
8    0.075333
1    0.073333
2    0.066000
6    0.066000
9    0.053333
5    0.048000
dtype: float64
